In [ ]:
# Importar las bibliotecas necesarias
import openai

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
from dotenv import load_dotenv

load_dotenv()

# Configurar la clave de API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")



# Retrieval Augmented Generation (RAG) con OpenAI

Retrieval Augmented Generation (RAG) es una técnica que combina la recuperación de información con la generación de texto para mejorar la calidad y relevancia de las respuestas generadas por modelos de lenguaje. En lugar de confiar únicamente en el modelo generativo para crear respuestas, RAG busca información relevante en una base de datos o repositorio de documentos para proporcionar contexto adicional al modelo.

Para simular un sistema de RAG, utilizamos un conjunto de 10 documentos generados.

In [ ]:
data_repository = [
    {
        "title": "IA en la Salud",
        "content": "La inteligencia artificial está transformando el sector de la salud al mejorar los diagnósticos, personalizar tratamientos y predecir resultados de los pacientes. Las herramientas de IA permiten a los médicos identificar patrones en grandes volúmenes de datos clínicos que serían difíciles de detectar manualmente."
    },
    {
        "title": "Impacto del Cambio Climático",
        "content": "El cambio climático está provocando el aumento del nivel del mar, incrementando la frecuencia de eventos climáticos extremos y afectando la biodiversidad global. Estas alteraciones están forzando a las comunidades a adaptarse y modificar sus prácticas agrícolas y de uso del suelo."
    },
    {
        "title": "Avances en Robótica",
        "content": "Los avances en robótica están habilitando la automatización en diversas industrias, mejorando la precisión en cirugías y aumentando la eficiencia en la fabricación. Los robots colaborativos están trabajando junto a los humanos en tareas complejas y repetitivas, reduciendo el margen de error."
    },
    {
        "title": "Computación Cuántica",
        "content": "La computación cuántica promete resolver problemas complejos más rápido que las computadoras tradicionales al utilizar principios de la mecánica cuántica. Esto podría revolucionar campos como la criptografía, la simulación de moléculas y la optimización de procesos industriales."
    },
    {
        "title": "Energías Renovables",
        "content": "El auge de las energías renovables está transformando el panorama energético global, impulsado por la necesidad de reducir las emisiones de carbono. La energía solar y eólica son líderes en esta transformación, con avances tecnológicos que están reduciendo costos y aumentando la eficiencia."
    },
    {
        "title": "Educación en la Era Digital",
        "content": "La educación está experimentando una revolución digital, con plataformas de aprendizaje en línea que ofrecen cursos accesibles a nivel mundial. La tecnología permite experiencias de aprendizaje personalizadas y accesibles, superando barreras geográficas y económicas."
    },
    {
        "title": "Tecnologías de la Información",
        "content": "La innovación en tecnologías de la información está impulsando cambios en la forma en que las empresas operan y se comunican. La computación en la nube, el big data y el internet de las cosas son pilares de la transformación digital, permitiendo una conectividad y análisis de datos sin precedentes."
    },
    {
        "title": "Biotecnología Moderna",
        "content": "La biotecnología moderna está permitiendo el desarrollo de terapias innovadoras, como la edición genética y las terapias basadas en células madre. Estas tecnologías están abriendo nuevas oportunidades para tratar enfermedades que anteriormente se consideraban incurables."
    },
    {
        "title": "Transporte Sostenible",
        "content": "El transporte sostenible está ganando impulso a medida que se busca reducir el impacto ambiental del transporte tradicional. Los vehículos eléctricos y la infraestructura de carga están evolucionando rápidamente, junto con nuevas formas de movilidad urbana como el carsharing y el ciclismo."
    },
    {
        "title": "Ciberseguridad",
        "content": "La ciberseguridad es una preocupación creciente a medida que las amenazas digitales se vuelven más sofisticadas. Las empresas están invirtiendo en tecnologías avanzadas de protección de datos y en la formación de sus empleados para mitigar riesgos y proteger la información crítica."
    },
]









Posteriormente tenemos un par de funciones para obtener los embeddings utilizando los modelos de OpenAI

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.embeddings.create(input = [text], model=model).data[0].embedding
                                   
def obtener_embeddings(textos):
    embeddings = [get_embedding(texto) for texto in textos]
    return embeddings


Llamamos las funciones para crear embeddings para el contenido en el repositorio de datos

In [ ]:
textos = [elemento["content"] for elemento in data_repository]
embeddings = obtener_embeddings(textos)

print(embeddings[2])


Búsqueda semántica - Función para recuperar el documento relevante utilizando cosine similarity

In [ ]:
def recuperar_documento_relevante(consulta, embeddings, data_repository):
    embedding_consulta = obtener_embeddings([consulta])[0]
    similitudes = cosine_similarity([embedding_consulta], embeddings)[0]
    indice_mejor_match = np.argmax(similitudes)
    return data_repository[indice_mejor_match]

Y una función para generar una respuesta usando RAG

In [ ]:
def generar_respuesta(consulta):
    # Recuperar documento relevante
    documento_relevante = recuperar_documento_relevante(consulta, embeddings, data_repository)
    contexto = documento_relevante["content"]

    # Usar OpenAI para generar una respuesta basada en el contexto y la consulta
    prompt = f"Contexto: {contexto}\n\n"+\
                f"Para la siguiente Consulta: {consulta}\n\n"+\
                "Genera una respuesta basada en el Contexto y solamente en el contexto, no uses la "+\
                "información que conoces solamente usa la información provista"
    print(f"PROMPT=\n=======\n{prompt}\n=======\n")
    respuesta = openai.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{prompt}"}
      ]
    )

    return respuesta.choices[0].message.content.strip()

Y ahora lo probamos

In [ ]:
# Consulta de ejemplo
consulta = "¿Cuáles son los impactos de la inteligencia artificial en la industria de la salud?"

# Generar respuesta usando RAG
respuesta = generar_respuesta(consulta)
print("Respuesta:", respuesta)